## DataLoader
torch.utils.data.DataLoader 概述</br>
DataLoader 是 PyTorch 中一个核心的实用程序，用于迭代数据集。它将 torch.utils.data.Dataset 对象（代表您的数据）封装起来，提供了一种便捷且高效的方式来：
</br>
1. 批处理 (Batching)： 将单个样本组合成小批次，以充分利用 GPU 的并行计算能力。</br>
2. 乱序 (Shuffling)： 在每个 epoch 开始时打乱数据，以减少模型训练的偏差并提高泛化能力。</br>
3. 多进程数据加载 (Multi-process Data Loading / num_workers)： 使用多个子进程并行加载数据，加速数据预处理和传输，防止数据加载成为训练的瓶颈（I/O 瓶颈）。</br>
4. 内存固定 (Pin Memory)： 将加载的数据存储在 CUDA 可直接访问的内存中，加速数据从 CPU 到 GPU 的传输。</br>

### 它常常包含以下几个组件：</br>
Sampler (索引生成器):DataLoader 不会直接调用 Dataset 的 __getitem__。</br>相反，它会使用一个 Sampler 来生成要获取的样本索引。</br>
SequentialSampler (默认): 按顺序生成 0 到 len(dataset)-1 的索引。</br>
RandomSampler (如果 shuffle=True): 在每个 epoch 开始时打乱索引并生成它们。</br>
BatchSampler (将索引聚合成批次): 将 Sampler 生成的单个索引聚合成批次的索引列表。DataLoader 默认会在内部使用 BatchSampler。</br>
DistributedSampler (分布式训练): 在分布式训练中，这个 Sampler 会确保每个 GPU 进程只处理数据集的一个不重叠的子集，并且可以进行乱序。</br>

collate_fn (批处理函数):</br>

当 DataLoader 从 Dataset 获取到单个样本的列表后（由 BatchSampler 提供的索引列表），它会调用 collate_fn 来将这些单个样本组合成一个批次的张量。</br>
默认的 collate_fn 会尝试将相同形状的张量堆叠（stack），并将 Python 数值类型转换为 PyTorch 张量。</br>
对于具有可变长度序列的数据（如 NLP 中的句子），您通常需要提供自定义的 collate_fn 来进行填充 (padding) 或其他特殊处理。</br>
num_workers (多进程加载):</br>

如果 num_workers > 0，DataLoader 会创建指定数量的子进程。</br>
每个子进程独立地执行 Dataset 的 __getitem__ 方法，并将加载到的数据发送回主进程。</br>
这显著加快了数据加载速度，因为 CPU 上的数据预处理和 I/O 可以与 GPU 上的模型训练并行进行。</br>
pin_memory (内存固定):</br>

如果 pin_memory=True 且在 CUDA 环境中，DataLoader 会将从 Dataset 获取到的数据存储在可由 CUDA 直接访问的 CPU 内存中（“pinned memory”或“page-locked memory”）。</br>
从 pinned memory 到 GPU 显存的数据传输比从常规 CPU 内存传输快得多。这减少了数据传输的延迟。</br>


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim


# --- 1. 定义一个简单的Dataset ---
# 这个Dataset将生成 y = 2*x + 1 加上一些噪声的数据
class SimpleLinearDataset(Dataset):
    def __init__(self, num_samples=1000):
        self.num_samples = num_samples
        # 生成 x 值，形状为 [num_samples, 1]
        self.X = torch.randn(num_samples, 1) * 10 # 随机生成一些数据，范围[-10, 10]左右
        # 生成 y 值，基于线性关系 y = 2*x + 1，并添加噪声
        self.y = 2 * self.X + 1 + torch.randn(num_samples, 1) * 2 # 添加少量噪声

    def __len__(self):
        """返回数据集的样本总数"""
        return self.num_samples

    def __getitem__(self, idx):
        """
        根据索引返回单个样本。
        DataLoader会多次调用这个方法来获取一个批次的数据。
        """
        return self.X[idx], self.y[idx]

In [ ]:
# --- 2. 定义一个简单的模型 (线性回归) ---
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        # 输入特征是1维，输出也是1维
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [ ]:
# --- 3. 设置训练参数 ---
num_samples = 1000 # 数据集总样本数
batch_size = 64    # 每个批次包含的样本数
num_epochs = 10    # 训练的轮数
learning_rate = 0.01

# 检查是否有GPU可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 4. 实例化Dataset和DataLoader ---
# 创建数据集实例
dataset = SimpleLinearDataset(num_samples=num_samples)

# 创建DataLoader实例
# shuffle=True: 在每个epoch开始时打乱数据，这对训练很重要。
# num_workers=0: 在单机单线程情况下，通常设置为0。如果数据预处理复杂或数据量非常大，可以考虑设置为 >0 来利用多核CPU并行加载。
# pin_memory=True: 如果使用GPU，设置为True可以加速数据从CPU到GPU的传输。
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

# --- 5. 实例化模型、损失函数和优化器 ---
model = LinearRegressionModel().to(device) # 将模型移到指定设备
criterion = nn.MSELoss() # 均方误差损失，适用于回归任务
optimizer = optim.SGD(model.parameters(), lr=learning_rate) # 使用随机梯度下降

# --- 6. 训练循环 ---
print("\nStarting training...")
loss_history = [] # 记录损失变化

for epoch in range(num_epochs):
    epoch_loss = 0.0
    # 迭代DataLoader，每次获取一个批次的数据
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        # 将数据移到GPU (如果可用)
        inputs = inputs.to(device)
        targets = targets.to(device)

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # 反向传播和优化
        optimizer.zero_grad() # 清零梯度
        loss.backward()       # 计算梯度
        optimizer.step()      # 更新模型参数

        epoch_loss += loss.item()

    avg_epoch_loss = epoch_loss / len(dataloader)
    loss_history.append(avg_epoch_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_epoch_loss:.4f}")

print("Training finished.")

Using device: cuda

Starting training...
Epoch [1/10], Average Loss: 246.2441
Epoch [2/10], Average Loss: 5.4720
Epoch [3/10], Average Loss: 6.8598
Epoch [4/10], Average Loss: 5.3417
Epoch [5/10], Average Loss: 4.2270
Epoch [6/10], Average Loss: 6.9698
Epoch [7/10], Average Loss: 4.0519
Epoch [8/10], Average Loss: 4.6585
Epoch [9/10], Average Loss: 4.0423
Epoch [10/10], Average Loss: 4.0123
Training finished.
